This file is intended to show a C-style implementation of Strassen's Algorithm for matrix multiplication.

Compare to Julia-style implementation, this is much more redundant.

We import `analyze.ipynb` to count the number of multiplication created by each of the two matrix multiplication implementations (naive implementation (called `BruteForce`) and Strassen's Algorithm).

In [1]:
# using Pkg
# Pkg.add("NBInclude")
using NBInclude

In [2]:
# import @analyze
@nbinclude("analyze.ipynb")

estimateComplexity (generic function with 2 methods)

In [3]:
# Define square matrix to avoid optimized brute force matrix multiplication
struct MySquareMatrix_Internal{T}
    A::Array{T,2}
    n::Int
end

In [4]:
# Define Constructor
MySquareMatrix(A::Array) = (length(size(A)) == 2 && size(A)[1] == size(A)[2]) ? MySquareMatrix_Internal(A,size(A)[1]) : throw(ArgumentError("A=$A is not a square matrix"))

MySquareMatrix (generic function with 1 method)

In [5]:
using Test

In [6]:
# Test Constructor
@test MySquareMatrix([1 2; 3 4]).A == [1 2; 3 4]

Test Passed

In [7]:
@test MySquareMatrix([1 2; 3 4]).n == 2

Test Passed

In [8]:
@test_throws ArgumentError MySquareMatrix([1 2 3; 3 4 5])

Test Passed
      Thrown: ArgumentError

In [9]:
# Define BruteForce implementation of matrix multiplication

function BruteForce(A::MySquareMatrix_Internal, B::MySquareMatrix_Internal)
    if typeof(A) != typeof(B)
        throw(ArgumentError("Two matrices should have the same type"))
    end
    if A.n != B.n
        throw(ArgumentError("Two matrices should have the same dimension"))
    end
    C = deepcopy(A)
    n = A.n
    for i=1:n
        for j=1:n
            C.A[i,j] = 0
            for k=1:n
                C.A[i,j] = C.A[i,j]+A.A[i,k]*B.A[k,j]
            end
        end
    end
    C
end

BruteForce (generic function with 1 method)

In [10]:
# Test BruteForce
A = MySquareMatrix([1 2; 3 4])

MySquareMatrix_Internal{Int64}([1 2; 3 4], 2)

In [11]:
B = MySquareMatrix([1 1; 1 1])

MySquareMatrix_Internal{Int64}([1 1; 1 1], 2)

In [12]:
BruteForce(A, B)

MySquareMatrix_Internal{Int64}([3 3; 7 7], 2)

In [13]:
# Define Common Operations on Matrix: Sum, Slice, Subtract
function Sum(A::MySquareMatrix_Internal, B::MySquareMatrix_Internal, )
    if typeof(A) != typeof(B)
        throw(ArgumentError("Two matrices should have the same type"))
    end
    if A.n != B.n
        throw(ArgumentError("Two matrices should have the same dimension"))
    end
    C = deepcopy(A)
    n = A.n
    for i=1:n
        for j=1:n
            C.A[i,j] += B.A[i,j]
        end
    end
    C
end

Sum (generic function with 1 method)

In [14]:
function Slice(A::MySquareMatrix_Internal,
        ARowStart::Int, ARowEnd::Int, AColStart::Int, AColEnd::Int)
    if (ARowEnd-ARowStart != AColEnd-AColStart)
        throw(ArgumentError("Matrix should be square matrices"))
    end
    C = zeros(Float64, ARowEnd-ARowStart+1, ARowEnd-ARowStart+1)
    MySquareMatrix(A.A[ARowStart:ARowEnd, AColStart:AColEnd])
end

Slice (generic function with 1 method)

In [15]:
function Subtract(A::MySquareMatrix_Internal, B::MySquareMatrix_Internal)
    if typeof(A) != typeof(B)
        throw(ArgumentError("Two matrices should have the same type"))
    end
    if A.n != B.n
        throw(ArgumentError("Two matrices should have the same dimension"))
    end
    C = deepcopy(A)
    n = A.n
    for i=1:n
        for j=1:n
            C.A[i,j] -= B.A[i,j]
        end
    end
    C
end

Subtract (generic function with 1 method)

In [16]:
@test Sum(MySquareMatrix([1 1; 1 1]), MySquareMatrix([1 1; 1 1])).A == [2 2; 2 2]

Test Passed

In [17]:
@test Subtract(MySquareMatrix([1 1; 1 1]), MySquareMatrix([1 1; 1 1])).A == [0 0; 0 0]

Test Passed

Then we define our Strassen's Algorithm. It consists of two parts: first is to pad zeros to make the original matrix into a square matrix of size 2^n.

In [18]:
function PadZeros(A::MySquareMatrix_Internal)
    n = A.n
    m = 2^(ceil(Int, log(2, n)))
    X = fill(0.0, m, m)
    for i=1:n
        for j=1:n
            X[i,j] = A.A[i,j]
        end
    end
    MySquareMatrix(X)
end

PadZeros (generic function with 1 method)

In [19]:
@test PadZeros(MySquareMatrix([1 2; 3 4])).A == [1 2; 3 4]

Test Passed

In [20]:
@test PadZeros(MySquareMatrix([1 2; 3 4])).n == 2

Test Passed

In [21]:
@test PadZeros(MySquareMatrix([1 1 1; 1 1 1; 1 1 1])).n == 4

Test Passed

In [22]:
# Define StrassenAlgo only for matrix of size power of 2, StrassenMultiply for general matrices
function SanityCheck(A::MySquareMatrix_Internal, B::MySquareMatrix_Internal)
    if typeof(A) != typeof(B)
        throw(ArgumentError("Two matrices should have the same type"))
    end
    if A.n != B.n
        throw(ArgumentError("Two matrices should have the same dimension"))
    end
    PadZeros(A), PadZeros(B)
end

function StrassenAlgo(A::MySquareMatrix_Internal, B::MySquareMatrix_Internal)
    # assume that input has been padded with zeros, and are of the same type & size
    n = A.n
    if n == 1
        return MySquareMatrix(fill(A.A[1, 1] * B.A[1, 1], 1, 1))
    end
    m = div(n, 2)
    A₁ = Slice(A, 1, m, 1, m)
    A₂ = Slice(A, 1, m, m+1, n)
    A₃ = Slice(A, m+1, n, 1, m)
    A₄ = Slice(A, m+1, n, m+1, n)
    B₁ = Slice(B, 1, m, 1, m)
    B₂ = Slice(B, 1, m, m+1, n)
    B₃ = Slice(B, m+1, n, 1, m)
    B₄ = Slice(B, m+1, n, m+1, n)
    M₁ = StrassenAlgo(Sum(A₁, A₄), Sum(B₁, B₄))
    M₂ = StrassenAlgo(Sum(A₃, A₄), B₁)
    M₃ = StrassenAlgo(A₁, Subtract(B₂, B₄))
    M₄ = StrassenAlgo(A₄, Subtract(B₃, B₁))
    M₅ = StrassenAlgo(Sum(A₁, A₂), B₄)
    M₆ = StrassenAlgo(Subtract(A₃, A₁), Sum(B₁, B₂))
    M₇ = StrassenAlgo(Subtract(A₂, A₄), Sum(B₃, B₄))
    C₁ = Sum(Sum(M₁, M₄), Subtract(M₇, M₅))
    C₂ = Sum(M₃, M₅)
    C₃ = Sum(M₂, M₄)
    C₄ = Sum(Sum(M₁, M₃), Subtract(M₆, M₂))
#     print(A₁.A, A₂.A, A₃.A, A₄.A, B₁.A, B₂.A, B₃.A, B₄.A, M₁.A, M₂.A, M₃.A, M₄.A, M₅.A, M₆.A, M₇.A, C₁.A, C₂.A, C₃.A, C₄.A)
    MySquareMatrix([C₁.A C₂.A ; C₃.A C₄.A ])
end

function StrassenMultiply(A::MySquareMatrix_Internal, B::MySquareMatrix_Internal)
    Slice(StrassenAlgo(SanityCheck(A, B)...), 1, A.n, 1, A.n)
end

StrassenMultiply (generic function with 1 method)

In [23]:
A = MySquareMatrix([1 2 3; 3 4 5; 5 6 7])

MySquareMatrix_Internal{Int64}([1 2 3; 3 4 5; 5 6 7], 3)

In [24]:
B = MySquareMatrix([4 4 5; 5 5 6; 5 6 7])

MySquareMatrix_Internal{Int64}([4 4 5; 5 5 6; 5 6 7], 3)

In [25]:
@test all(StrassenMultiply(A, B).A.== BruteForce(A, B).A)

Test Passed

In [26]:
# @analyze StrassenMultiply(A, B)

We try to conduct some analyses on these two implementations. Yet, because neither implementation is optimized or well-written, these results are not very informative. For better analysis, see `Identities.ipynb`.

In [27]:
@analyze BruteForce(A, B)

Number of additions: 58
Number of subtractions: 59
Number of multiplications: 28
Number of divisions: 0


In [31]:
@analyze StrassenMultiply(A, B)

Number of additions: 2688
Number of subtractions: 3238
Number of multiplications: 259
Number of divisions: 6


In [28]:
X = MySquareMatrix(fill(1, 16, 16))

MySquareMatrix_Internal{Int64}([1 1 … 1 1; 1 1 … 1 1; … ; 1 1 … 1 1; 1 1 … 1 1], 16)

In [32]:
@analyze BruteForce(X, X)

Number of additions: 8196
Number of subtractions: 4648
Number of multiplications: 4097
Number of divisions: 0


In [29]:
@analyze StrassenMultiply(X, X)

Number of additions: 144235
Number of subtractions: 168765
Number of multiplications: 11237
Number of divisions: 6
